<img src="logo.png">

Vamos a usar el dataset del [Titanic](https://www.kaggle.com/c/titanic/data), que contiene los datos de los pasajeros del Titanic, famoso barco que se hundió en su primer viaje en 1912. Dicho dataset contiene los datos de los pasajeros así como información sobre si sobrevivieron o no.

**Descripción de los datos**

* superviviente 	 El pasajero sobrevivió 	0 = No, 1 = Si

* clase_billete 	 Clase de camarote 	1 = Primera clase, 2 = Segunda, 3 = Tercera clase (la más pobre)

* genero 	         hombre/mujer 	

* edad 	

* n_hermanos_esposos Número de hermanos o pareja a bordo del Titanic

* n_hijos_padres     Número de hijos o padres a bordo del Titanic 

* precio_billete 	

* puerto_salida      Puerto donde el pasajero tomó el barco (C=Cherbourg, Q=Queenstown, S=Southampton)


In [ ]:
import pandas as pd

In [ ]:
datos = pd.read_csv("titanic.csv")

In [ ]:
datos.head()

Los algoritmos de creación de árboles están en el submódulo de `sklearn.tree` 

En cuanto al tipo de algoritmo para crear árboles, scikit-learn [usa una versión optimizada del algoritmo CART](http://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart) *(Classification and Regression Trees)*, que  permite usar árboles de decisión tanto para problemas de clasificación como de regresión.

In [ ]:
from sklearn import tree
from sklearn.model_selection import cross_val_score

In [ ]:
arbol = tree.DecisionTreeClassifier()

In [ ]:
from sklearn import preprocessing

In [ ]:
columnas_categoricas = ["genero", "puerto_salida"]

In [ ]:
datos_categoricos = pd.get_dummies(datos[columnas_categoricas])

In [ ]:
datos_categoricos

In [ ]:
pasajeros = (
    pd.concat([
        datos.drop(columnas_categoricas, axis=1),
        datos_categoricos
    ],axis=1
    )
)
pasajeros.edad = pasajeros.edad.fillna(pasajeros.edad.mean())

In [ ]:
pasajeros.head()

In [ ]:
arbol.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

In [ ]:
cross_val_score(arbol, pasajeros.drop("superviviente", axis=1), pasajeros.superviviente, scoring="roc_auc", 
                cv=10).mean()

Una funcionalidad interesante que tienen los arboles de decision en `sklearn` es que se pueden visualizar con `Graphviz`. Para ello tenemos que instalar la libreria con `pip install graphviz`.

In [ ]:
%matplotlib inline

In [ ]:
!pip install graphviz

Se puede exportar el árbol y abrirlo posteriormente con graphviz desde la terminal (o desde la página http://webgraphviz.com/ que renderiza archivos de graphviz)

In [ ]:
import graphviz

tree.export_graphviz(arbol, out_file="arbol.dot")

Otra funcionalidad que tienen los árboles de decisión en sklearn es que nos dan una indicación de la importancia de cada variable en el modelo, almacenada en el atributo `feature_importances_`. Calcula la importancia en función de la ganancia de información de cada variable, es decir, que variables separan mejor las distintas clases.

In [ ]:
arbol.feature_importances_

In [ ]:
dict(zip(
    pasajeros.drop("superviviente", axis=1),
    arbol.feature_importances_
))

Por ejemplo, en este árbol vemos que las variables que tienen más peso a la hora de decidir si un pasajero sobrevive o no son la edad, el género y el precio del billete (¡las mujeres y los niños primero!).

In [ ]:
tree.DecisionTreeClassifier?

Éstos son los parámetros más importantes para los modelos DecisionTreeClassifier de sklearn:

* criterion : El criterio para calcular la reducción de impureza (ganancia de información) al hacer una partición. Se puede elegir entre `gini`, o `entropy` 

* max_depth : La profundidad máxima del árbol. Definimos profundidad como el número de nodos que atraviesa una observación *(cuantas "preguntas" se le hacen)*.

* max_features:  El máximo numero de particiones potenciales se consideran al evaluar un nodo.

* max_leaf_nodes : Límite de hojas para el árbol.

* min_impurity_decrease : la ganancia de información mínima en un nodo para hacer una partición. (Si no hay ninguna partición que cumpla este criterio, se para el desarrollo del árbol en dicho nodo).

* class_weight : Para clases imbalanceadas, podemos pasar el argumento class_weight, como un diccionario de `{clase: peso}` para que sklearn tenga en cuenta los pesos. Alternativamente, podemos pasar el string `balanced` para que sklearn genere pesos en función del número de muestras de cada clase.

Los árboles de decisión tienden a sobreajustar cuando tienen demasiada complejidad. Los siguientes parámetros ayudan a controlarla:

* min_samples_leaf : Mínimo número de observaciones en un nodo para considerar un nodo como una hoja. Por defecto es 1, ésto significa que sklearn solo para de hacer particiones cuando un nodo tiene sólo 1 observación.

* min_samples_split : Mínimo número de observaciones en un nodo para generar una partición. Por defecto es 2, esto significa que sklearn por defecto siempre va a intentar particionar un nodo con 2+ elementos.

In [ ]:
arbol_simple = tree.DecisionTreeClassifier(max_depth=3)

In [ ]:
arbol_simple.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

In [ ]:
tree.export_graphviz(arbol_simple, out_file="arbol_simple.dot")

In [ ]:
cross_val_score(arbol_simple, pasajeros.drop("superviviente", axis=1), 
                pasajeros.superviviente, scoring="roc_auc", cv=10).mean()

In [ ]:
arbol_balanceado = tree.DecisionTreeClassifier(max_depth=3, class_weight="balanced")

In [ ]:
arbol_balanceado.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

In [ ]:
tree.export_graphviz(arbol_balanceado, out_file="arbol_balanceado.dot")

In [ ]:
cross_val_score(arbol_balanceado, pasajeros.drop("superviviente", axis=1), 
                pasajeros.superviviente, scoring="roc_auc", cv=10).mean()

In [ ]:
cross_val_score(arbol_balanceado, pasajeros.drop("superviviente", axis=1), pasajeros.superviviente,
                scoring="precision", cv=10).mean()

Además del algoritmo CART para generar árboles, scikit-learn también proporciona una clase de arboles llamada ExtraTreeClassifier, o Extremely Random Trees (Árboles Extremadamente Aleatorios). En estos árboles, en lugar de seleccionar en cada nodo la párticion que proporciona la mayor ganancia de información, ¡se decide una partición al azar!.

In [ ]:
arbol_aleatorio = tree.ExtraTreeClassifier(max_features=1)

In [ ]:
arbol_aleatorio.fit(pasajeros.drop("superviviente", axis=1), pasajeros.superviviente)

tree.export_graphviz(arbol_aleatorio, out_file="arbol_aleatorio.dot")

Los árboles aleatorios funcionan peor que los árboles de decisión por sí mismos.

In [ ]:
cross_val_score(arbol_aleatorio, pasajeros.drop("superviviente", axis=1), pasajeros.superviviente, scoring="roc_auc",
                cv=10).mean()